In [1]:
def reference_genome():
    #Coronavirus_2_isolate_genome.fasta
    #Set script to look in reference_genome folder. 

    from pathlib import Path

    
    path=Path("reference_genome/")
    file_name=''
    for entry in path.iterdir():
        print("*"*100)
        print("\nReference Genome Used:\n '" + entry.name+ "'\n" )
        file_name=entry.name
        break
        

    
    #Open File
    fh=open("reference_genome/"+file_name, 'r')
    fn=fh.readlines()
    corona_genome=''
    for line in fn:
        if line.startswith(">"):
            continue
        else:
            corona_genome+=line.strip()
    return corona_genome
    
def read_bed_file():
    import pandas as pd
    from pathlib import Path
    file_list=[]
    path=Path("bed/")
    file_name=''

    for entry in path.iterdir():
        
        file_name=entry.name
        save_file= file_name.strip(".bed")
        bed_file=open("bed/"+file_name, "r")
        
        #On MacOS filters the .DS_Store file. 
        if file_name.startswith("."):
            continue
        else: 
            
            bed_file_list=bed_file.readlines()

            yield bed_file_list,save_file
        
        
        #print("bed file found :'" + entry.name+"'" )
        #file_list.append(entry.name)

def create_dataframe(a_list,corona_genome):
    
    import pandas as pd
    from pathlib import Path
    root="output_vcf/"
    new_file=[]
    print("*"*100)
    print("Converting File: "+ a_list[1]+".bed\n")
    print("Indels found: ")
    path=Path("bed/") 
    for line in a_list[0]:

        df_list=line.split("\t")
        ID=df_list[len(df_list)-1].strip()
        deletion_start=int(df_list[len(df_list)-3].strip())-1
        deletion_end=int(df_list[len(df_list)-2].strip())
        #print(deletion_start,deletion_end)
        if ID.startswith("d") or ID.startswith("i"):


            #Sets pos equal to pos-1
            #to follow the sample vcf file. 
            b=line.split()
            b[2]=b[1]
            b='\t'.join(b)
            #print(b)


            original=b.strip()


            original+="\t" +corona_genome[deletion_start:deletion_end]+"\t"+corona_genome[deletion_start]

            new_file.append(original)



            print(" •",ID[len(ID)-len(ID):] ,": ",corona_genome[deletion_start:deletion_end])


        else:    
            original=line.strip()
            original+="\t" +ID[len(ID)-len(ID)]+"\t"+ID[len(ID)-1]
            new_file.append(original)

        
    df = pd.DataFrame([sub.split("\t") for sub in new_file],index=None,columns=["#CHROM", "POS-1", "POS", 'ID',"REF", "ALT"])
    df.drop('POS-1', inplace=True, axis=1)
    
    df.to_csv(root+a_list[1]+".vcf",sep="\t",header=True, index=False)    
    print("\nPreview:  " +a_list[1]+ ".vcf\n\n",df,"\n")


    


    



In [9]:
def main():
    import pandas as pd
    from pathlib import Path
    reference=reference_genome()
    
    print("Converting Files: ")
    path=Path("bed/")
    
    #print files info
    for entry in path.iterdir():
        if entry.name.startswith("."):
            continue
        else:
            print(entry.name)
        
    
    
    
    #Use Generator
    for i in read_bed_file():
        create_dataframe(i,reference)
    
    print(".vcf files saved in '/output_vcf' directory.")
if __name__ == "__main__":
    main()

****************************************************************************************************

Reference Genome Used:
 'Coronavirus_2_isolate_genome.fasta'

Converting Files: 
P.1.bed
B_1_351.bed
P_1.bed
B_1_429.bed
B.1.429.bed
****************************************************************************************************
Converting File: P.1.bed

Indels found: 
 • del_11288 :  GTCTGGTTTT
 • ins_28263 :  GA

Preview:  P.1.vcf

          #CHROM    POS         ID         REF ALT
0   NC_045512v2    241      C241T           C   T
1   NC_045512v2    733      T733C           T   C
2   NC_045512v2   2749     C2749T           C   T
3   NC_045512v2   3037     C3037T           C   T
4   NC_045512v2   3828     C3828T           C   T
5   NC_045512v2   5648     A5648C           A   C
6   NC_045512v2   6319     A6319G           A   G
7   NC_045512v2   6613     A6613G           A   G
8   NC_045512v2  11287  del_11288  GTCTGGTTTT   G
9   NC_045512v2  12778    C12778T           C   T
10  NC

In [1]:
"GTCTGGTTTT" == "GTCTGGTTTT"

True

In [10]:
! cat /Users/juanbolanos/Desktop/Nanopore_Pipeline/00_Compare_Variants/01_Consensus_Sequences/consensus_VCF/UNCC-6.pass.vcf

##fileformat=VCFv4.2
##nanopolish_window=MN908947.3:1-29902
##artic-tools_version=0.2.6
##INFO=<ID=TotalReads,Number=1,Type=Integer,Description="The number of event-space reads used to call the variant">
##INFO=<ID=SupportFraction,Number=1,Type=Float,Description="The fraction of event-space reads that support the variant">
##INFO=<ID=SupportFractionByStrand,Number=2,Type=Float,Description="Fraction of event-space reads that support the variant for each strand">
##INFO=<ID=BaseCalledReadsWithVariant,Number=1,Type=Integer,Description="The number of base-space reads that support the variant">
##INFO=<ID=BaseCalledFraction,Number=1,Type=Float,Description="The fraction of base-space reads that support the variant">
##INFO=<ID=AlleleCount,Number=1,Type=Integer,Description="The inferred number of copies of the allele">
##INFO=<ID=StrandSupport,Number=4,Type=Integer,Description="Number of reads supporting the REF and ALT allele, by strand">
##INFO=<ID=StrandFisherTest,Number=1,Type=Integer,Des

In [7]:
! bedtools intersect -a /users/juanbolanos/Desktop/bed_to_vcf/DNA_bed_to_vcf/bed/B.1.429.bed  -b /users/juanbolanos/Desktop/bed_to_vcf/AA_bed_to_vcf/bed/B.1.429_AAmuts.bed

NC_045512v2	1058	1059	C1059T
NC_045512v2	14407	14408	C14408T
NC_045512v2	17013	17014	G17014T
NC_045512v2	21599	21600	G21600T
NC_045512v2	22017	22018	G22018T
NC_045512v2	22916	22917	T22917G
NC_045512v2	23402	23403	A23403G
NC_045512v2	25562	25563	G25563T
NC_045512v2	28886	28887	C28887T


In [5]:
! ls /users/juanbolanos/Desktop/bed_to_vcf/DNA_bed_to_vcf/bed/

B.1.429.bed B_1_351.bed B_1_429.bed P.1.bed     P_1.bed


In [6]:
! ls /users/juanbolanos/Desktop/bed_to_vcf/AA_bed_to_vcf/bed/

B.1.1.7_AAmuts.bed B.1.351_AAmuts.bed B.1.429_AAmuts.bed P.1_AAmuts.bed
